In [ ]:
%history -g -f jup_history

## Precipitation Products

In [ ]:
## Precipitation Datasets
gridMET = 'IDAHO_EPSCOR/GRIDMET',      variable = 'pr' (mm/day)
DAYMET = 'NASA/ORNL/DAYMET_V4',        variable = 'prcp' (mm/day)
NLDAS = 'NASA/NLDAS/FORA0125_H002',    variable = 'total_precipitation' (kg/m^2) hourly
PRISM = 'OREGONSTATE/PRISM/AN81m',     variable = 'ppt'	(mm/month)
IMERG = 'NASA/GPM_L3/IMERG_MONTHLY_V06', variable = 'precipitation' (mm/hr) monthly

In [1]:
import ee

# # Uncomment to Authenticate and Initialize the Earth Engine Python API
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1Adeu5BW1RYEvPfjohZtxb-TAj-_jQzxuLigpSqDNKo2hX4PcvfU3LZb92bM



Successfully saved authorization token.


## PRISM

In [1]:
import ee

# Uncomment to Authenticate and Initialize the Earth Engine Python API
#ee.Authenticate()
ee.Initialize()

# Parameters
filterdate = 2016
startDate = ee.Date.fromYMD(filterdate, 1, 1)  # inclusive
endDate = ee.Date.fromYMD(filterdate+1, 1, 1)  # exclusive
# Feature Dataset
combinedFeatureCollection = ee.FeatureCollection('projects/uom-project-0-1/assets/HPA_Irr_Croplands_2016')

# Output some info
print('Size of combinedFeatureCollection', combinedFeatureCollection.size().getInfo())

# Subset of Polygons
numberofPolygons = combinedFeatureCollection.size().getInfo()
Sub_FeatureCollection = ee.FeatureCollection(combinedFeatureCollection.toList(numberofPolygons))

# Precipitation Datasets
prDatasets = [
    'OREGONSTATE/PRISM/AN81m'
            ]

def extract_feature(feature, dataset):
    geom = feature.geometry()

    def extract_image(image):
        image = image.unmask()
        obj = image.reduceRegion(combinedreducers, geom, 30)  # 90m
        return feature.setMulti(obj)

    return dataset.filterBounds(geom).map(extract_image)

combinedreducers = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(),
    sharedInputs=True
).combine(
    reducer2=ee.Reducer.sum(),
    sharedInputs=True
)

for dataset in prDatasets:
    # Precipitation dataset
    pr = ee.ImageCollection(dataset) \
        .filterDate(startDate, endDate) \
        .select('ppt')

    Sub_FeatureCollection_pr = Sub_FeatureCollection.map(lambda feature: extract_feature(feature, pr)).flatten()
    Sub_FeatureCollection_pr = Sub_FeatureCollection_pr.select(['.*'], None, False)

    # Output configuration
    #link = str(startDate.split('-')[-3] + '_' + endDate.split('-')[-3])
    description = dataset.split('/')[-2] + f'_PRC_{filterdate}'
    output_folder = '001_Project_Exports'
    print(description)

    # Export as CSV to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=Sub_FeatureCollection_pr,
        description=description,
        folder=output_folder,
        fileNamePrefix=description,
        fileFormat='CSV'
    )
    task.start()

Size of combinedFeatureCollection 114872
PRISM_PRC_2016


## NLDAS

In [4]:
import ee

# Uncomment to Authenticate and Initialize the Earth Engine Python API
#ee.Authenticate()
ee.Initialize()

# Parameters
filterdate = 2019
startDate = ee.Date.fromYMD(filterdate, 1, 1)  # inclusive
endDate = ee.Date.fromYMD(filterdate+1, 1, 1)  # exclusive
# Feature Dataset
combinedFeatureCollection = ee.FeatureCollection('projects/uom-project-0-1/assets/HPA_Irr_Croplands_2019')

## Output some info
print('Size of combinedFeatureCollection', combinedFeatureCollection.size().getInfo())

# Batch size for processing polygons
batch_size = 4000  # Adjust this value as needed

# Get the total number of polygons
total_polygons = combinedFeatureCollection.size().getInfo()

# Calculate the number of batches required
num_batches = total_polygons // batch_size + (1 if total_polygons % batch_size != 0 else 0)
print(num_batches)

# Create a nested list of feature collections
feature_collection_list = ee.List.sequence(0, total_polygons, batch_size).map(
    lambda start_index: combinedFeatureCollection.toList(batch_size, start_index)
)

# Precipitation Datasets
prDatasets = [
    'NASA/NLDAS/FORA0125_H002'
]

# Process polygons in batches
for i in range(num_batches):
    Sub_FeatureCollection = ee.FeatureCollection(ee.List(feature_collection_list.get(i)).flatten())
    #print(Sub_FeatureCollection, f"Sub_FeatureCollection (Batch {i+1}/{num_batches})")

    def extract_feature(feature, dataset):
        geom = feature.geometry()

        def extract_image(image):
            image = image.unmask()
            obj = image.reduceRegion(combinedreducers, geom, 30)  # 90m
            return feature.setMulti(obj)

        return dataset.filterBounds(geom).map(extract_image)

    combinedreducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.max(),
        sharedInputs=True
    )

    for dataset in prDatasets:
        # Precipitation dataset
        pr = ee.ImageCollection(dataset) \
            .filterDate(startDate, endDate) \
            .select('total_precipitation')
        
        # Calculate dailyDataset using the provided function
        difdate = endDate.advance(-1, 'day').difference(startDate, 'day')
        createList = ee.List.sequence(0, difdate)
        listdates = createList.map(lambda day: startDate.advance(day, 'day'))

        # Daily precipitation dataset
        dailyDataset = ee.ImageCollection.fromImages(listdates.map(lambda summarize_day:
            pr.filterDate(ee.Date(summarize_day), ee.Date(summarize_day).advance(1, 'day'))
              .sum().select(['total_precipitation'])
              .copyProperties(pr.first()).setMulti({
                    'Date': ee.Date(summarize_day),
                    'system:time_start': ee.Date(summarize_day).millis()
                })
        ))

        Sub_FeatureCollection_pr = Sub_FeatureCollection.map(lambda feature: extract_feature(feature, dailyDataset)).flatten()
        Sub_FeatureCollection_pr = Sub_FeatureCollection_pr.select(['.*'], None, False)

        # Output configuration
        #link = str(startDate.split('-')[-3] + '_' + endDate.split('-')[-3])
        description = f'NLDAS_PRC_Batch_{i+1}_{filterdate}'
        output_folder = '001_Project_Exports'
        print(description)

        # Export as CSV to Google Drive
        task = ee.batch.Export.table.toDrive(
            collection=Sub_FeatureCollection_pr,
            description=description,
            folder=output_folder,
            fileNamePrefix=description,
            fileFormat='CSV'
        )
        task.start()
print("Done!")

Size of combinedFeatureCollection 114872
29
NLDAS_PRC_Batch_1_2019
NLDAS_PRC_Batch_2_2019
NLDAS_PRC_Batch_3_2019
NLDAS_PRC_Batch_4_2019
NLDAS_PRC_Batch_5_2019
NLDAS_PRC_Batch_6_2019
NLDAS_PRC_Batch_7_2019
NLDAS_PRC_Batch_8_2019
NLDAS_PRC_Batch_9_2019
NLDAS_PRC_Batch_10_2019
NLDAS_PRC_Batch_11_2019
NLDAS_PRC_Batch_12_2019
NLDAS_PRC_Batch_13_2019
NLDAS_PRC_Batch_14_2019
NLDAS_PRC_Batch_15_2019
NLDAS_PRC_Batch_16_2019
NLDAS_PRC_Batch_17_2019
NLDAS_PRC_Batch_18_2019
NLDAS_PRC_Batch_19_2019
NLDAS_PRC_Batch_20_2019
NLDAS_PRC_Batch_21_2019
NLDAS_PRC_Batch_22_2019
NLDAS_PRC_Batch_23_2019
NLDAS_PRC_Batch_24_2019
NLDAS_PRC_Batch_25_2019
NLDAS_PRC_Batch_26_2019
NLDAS_PRC_Batch_27_2019
NLDAS_PRC_Batch_28_2019
NLDAS_PRC_Batch_29_2019
Done!


## returns month and year

In [21]:
import ee

# Initialize the Earth Engine Python API
ee.Initialize()

# Parameters
filterdate = 2016
startDate = ee.Date.fromYMD(filterdate, 1, 1)  # inclusive
endDate = ee.Date.fromYMD(filterdate+1, 1, 1)  # exclusive

# Feature Dataset
combinedFeatureCollection = ee.FeatureCollection('projects/uom-project-0-1/assets/HPA_Irr_Croplands_2016')

# Output some info
print('Size of combinedFeatureCollection', combinedFeatureCollection.size().getInfo())

# Subset of Polygons
numberofPolygons = 2
Sub_FeatureCollection = ee.FeatureCollection(combinedFeatureCollection.toList(numberofPolygons))
#print(Sub_FeatureCollection, "Sub_FeatureCollection")

# Precipitation Dataset
pr = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
    .filterDate(startDate, endDate) \
    .select('prcp')

# Do some quick checks
#print('Size of Precipitation Collection', pr.size())
#print(pr.first(), "example precipitation image")

# Extract Precipitation values for each location for csv output
def extract_feature(feature):
    geom = feature.geometry()

    def extract_image(image):
        image = image.unmask()
        date = ee.Date(image.get('system:time_start'))
        year = date.get('year')
        month = date.get('month')
        obj = image.reduceRegion(ee.Reducer.sum(), geom, 30)  # 90m
        return feature.setMulti(obj).set('year', year).set('month', month)

    return pr.filterBounds(geom).map(extract_image)

Sub_FeatureCollection_pr = Sub_FeatureCollection.map(extract_feature).flatten()

# # Update system:index column
Sub_FeatureCollection_pr = Sub_FeatureCollection_pr.map(lambda feature: feature.set('system:index', ee.String(feature.get('system:index')).split('_').get(1)))

# #print(Sub_FeatureCollection_pr.first(), "Sub_FeatureCollection_pr Example")

# Monthly Total Precipitation
monthlyPrecipitation = Sub_FeatureCollection_pr.reduceColumns(ee.Reducer.sum().group(1, 'monthly_total'), ['month', 'prcp'])

# Output configuration
link = str(filterdate)
description = 'DAYMET_Prcp_1to5_2016'
output_folder = '001_Project_Exports'

# Export as CSV to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=Sub_FeatureCollection_pr,
    description=description,
    folder=output_folder,
    fileNamePrefix=description,
    fileFormat='CSV'
)
task.start()


Size of combinedFeatureCollection 129482


## Batch downloading for gridMET and DAYMET

In [5]:
import ee

# Uncomment to Authenticate and Initialize the Earth Engine Python API
#ee.Authenticate()
ee.Initialize()

# Parameters
filterdate = 2016
startDate = ee.Date.fromYMD(filterdate, 1, 1)  # inclusive
endDate = ee.Date.fromYMD(filterdate+1, 1, 1)  # exclusive
# Feature Dataset
combinedFeatureCollection = ee.FeatureCollection('projects/uom-project-0-1/assets/HPA_Irr_Croplands_2016')

## Output some info
print('Size of combinedFeatureCollection', combinedFeatureCollection.size().getInfo())

# Batch size for processing polygons
batch_size = 4000  # Adjust this value as needed

# Get the total number of polygons
total_polygons = combinedFeatureCollection.size().getInfo()

# Calculate the number of batches required
num_batches = total_polygons // batch_size + (1 if total_polygons % batch_size != 0 else 0)
print(num_batches)

# Create a nested list of feature collections
feature_collection_list = ee.List.sequence(0, total_polygons, batch_size).map(
    lambda start_index: combinedFeatureCollection.toList(batch_size, start_index)
)

# Precipitation Datasets
prDatasets = [
    'NASA/ORNL/DAYMET_V4'
    ]

# Process polygons in batches
for i in range(num_batches):
    Sub_FeatureCollection = ee.FeatureCollection(ee.List(feature_collection_list.get(i)).flatten())
    #print(Sub_FeatureCollection, f"Sub_FeatureCollection (Batch {i+1}/{num_batches})")

    def extract_feature(feature, dataset):
        geom = feature.geometry()

        def extract_image(image):
            image = image.unmask()
            obj = image.reduceRegion(combinedreducers, geom, 30)  # 90m
            return feature.setMulti(obj)

        return dataset.filterBounds(geom).map(extract_image)

    combinedreducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.max(),
        sharedInputs=True
    )

    for dataset in prDatasets:
        # Precipitation dataset
        pr = ee.ImageCollection(dataset) \
            .filterDate(startDate, endDate) \
            .select('prcp')

        Sub_FeatureCollection_pr = Sub_FeatureCollection.map(lambda feature: extract_feature(feature, pr)).flatten()
        Sub_FeatureCollection_pr = Sub_FeatureCollection_pr.select(['.*'], None, False)

        # Output configuration
        #link = str(startDate.split('-')[-3] + '_' + endDate.split('-')[-3])
        description = f'DAYMET_PRC_Batch_{i+1}_{filterdate}'
        output_folder = '001_Project_Exports'
        print(description)

        # Export as CSV to Google Drive
        task = ee.batch.Export.table.toDrive(
            collection=Sub_FeatureCollection_pr,
            description=description,
            folder=output_folder,
            fileNamePrefix=description,
            fileFormat='CSV'
        )
        task.start()
print('Done!')

Size of combinedFeatureCollection 114872
29
DAYMET_PRC_Batch_1_2016
Done!
DAYMET_PRC_Batch_2_2016
Done!
DAYMET_PRC_Batch_3_2016
Done!
DAYMET_PRC_Batch_4_2016
Done!
DAYMET_PRC_Batch_5_2016
Done!
DAYMET_PRC_Batch_6_2016
Done!
DAYMET_PRC_Batch_7_2016
Done!
DAYMET_PRC_Batch_8_2016
Done!
DAYMET_PRC_Batch_9_2016
Done!
DAYMET_PRC_Batch_10_2016
Done!
DAYMET_PRC_Batch_11_2016
Done!
DAYMET_PRC_Batch_12_2016
Done!
DAYMET_PRC_Batch_13_2016
Done!
DAYMET_PRC_Batch_14_2016
Done!
DAYMET_PRC_Batch_15_2016
Done!
DAYMET_PRC_Batch_16_2016
Done!
DAYMET_PRC_Batch_17_2016
Done!
DAYMET_PRC_Batch_18_2016
Done!
DAYMET_PRC_Batch_19_2016
Done!
DAYMET_PRC_Batch_20_2016
Done!
DAYMET_PRC_Batch_21_2016
Done!
DAYMET_PRC_Batch_22_2016
Done!
DAYMET_PRC_Batch_23_2016
Done!
DAYMET_PRC_Batch_24_2016
Done!
DAYMET_PRC_Batch_25_2016
Done!
DAYMET_PRC_Batch_26_2016
Done!
DAYMET_PRC_Batch_27_2016
Done!
DAYMET_PRC_Batch_28_2016
Done!
DAYMET_PRC_Batch_29_2016
Done!


## IMERG

In [13]:
import ee

# Uncomment to Authenticate and Initialize the Earth Engine Python API
#ee.Authenticate()
ee.Initialize()

# Parameters
filterdate = 2021
startDate = ee.Date.fromYMD(filterdate, 1, 1)  # inclusive
endDate = ee.Date.fromYMD(filterdate+1, 1, 1)  # exclusive
# Feature Dataset
combinedFeatureCollection = ee.FeatureCollection('projects/uom-project-0-1/assets/HPA_Irr_Croplands_2021')

# Output some info
print('Size of combinedFeatureCollection', combinedFeatureCollection.size().getInfo())

# Subset of Polygons
numberofPolygons = combinedFeatureCollection.size().getInfo()
Sub_FeatureCollection = ee.FeatureCollection(combinedFeatureCollection.toList(numberofPolygons))

# Precipitation Dataset
prDatasets = [
    'NASA/GPM_L3/IMERG_MONTHLY_V06'
]

# below function return number of days in a month
def getDaysInMonth(y, m):
    dt = ee.Date.fromYMD(y, m, 1)
    n = dt.advance(1, "month").difference(dt, 'day')
    return n

# below function will convert mm/hr to mm/month for the GPM data
def monthly(image):
    dt = ee.Date(image.get("system:time_end"))
    y = dt.get('year')
    m = dt.get('month')
    days = getDaysInMonth(y, m)
    return image.multiply(days).multiply(24).set({
        "system:time_start": image.get("system:time_start"),
        "system:time_end": image.get("system:time_end")
    })

def extract_feature(feature, dataset):
    geom = feature.geometry()

    def extract_image(image):
        image = image.unmask()
        obj = image.reduceRegion(combinedreducers, geom, 30)  # 90m
        return feature.setMulti(obj)

    return dataset.filterBounds(geom).map(extract_image)

combinedreducers = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(),
    sharedInputs=True
).combine(
    reducer2=ee.Reducer.sum(),
    sharedInputs=True
)

for dataset in prDatasets:
    # Precipitation dataset
    pr = ee.ImageCollection(dataset) \
        .filterDate(startDate, endDate) \
        .select('precipitation')

    pr_monthly = pr.map(monthly)  # Applying the monthly function

    Sub_FeatureCollection_pr = Sub_FeatureCollection.map(lambda feature: extract_feature(feature, pr_monthly)).flatten()
    Sub_FeatureCollection_pr = Sub_FeatureCollection_pr.select(['.*'], None, False)

    # Output configuration
    description = dataset.split('/')[-1] + f'_PRC_{filterdate}'
    output_folder = '001_Project_Exports'
    print(description)

    # Export as CSV to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=Sub_FeatureCollection_pr,
        description=description,
        folder=output_folder,
        fileNamePrefix=description,
        fileFormat='CSV'
    )
    task.start()

Size of combinedFeatureCollection 114872
IMERG_MONTHLY_V06_PRC_2021


## Download only one batch NLDAS

In [3]:
import ee

# Uncomment to Authenticate and Initialize the Earth Engine Python API
# ee.Authenticate()
ee.Initialize()

# Parameters
filterdate = 2019
startDate = ee.Date.fromYMD(filterdate, 1, 1)  # inclusive
endDate = ee.Date.fromYMD(filterdate+1, 1, 1)  # exclusive
# Feature Dataset
combinedFeatureCollection = ee.FeatureCollection('projects/uom-project-0-1/assets/HPA_Irr_Croplands_2019')

# Batch size for processing polygons
batch_size = 4000  # Adjust this value as needed

# Get the total number of polygons
total_polygons = combinedFeatureCollection.size().getInfo()

# Calculate the number of batches required
num_batches = total_polygons // batch_size + (1 if total_polygons % batch_size != 0 else 0)
print(num_batches)

# Specify the batch you want to process (0-indexed)
target_batch = 9

# Create a nested list of feature collections
feature_collection_list = ee.List.sequence(0, total_polygons, batch_size).map(
    lambda start_index: combinedFeatureCollection.toList(batch_size, start_index)
)

# Precipitation Datasets
prDatasets = [
    'NASA/NLDAS/FORA0125_H002'
]

# Process only batch 9
for i in range(num_batches):
    if i == target_batch:
        Sub_FeatureCollection = ee.FeatureCollection(ee.List(feature_collection_list.get(i)).flatten())
        
        # Define the extract_feature function and combinedreducers here
        
        for dataset in prDatasets:
            # Precipitation dataset
            pr = ee.ImageCollection(dataset) \
                .filterDate(startDate, endDate) \
                .select('total_precipitation')
            
            # Calculate dailyDataset using the provided function
            difdate = endDate.advance(-1, 'day').difference(startDate, 'day')
            createList = ee.List.sequence(0, difdate)
            listdates = createList.map(lambda day: startDate.advance(day, 'day'))
            
            # Daily precipitation dataset
            dailyDataset = ee.ImageCollection.fromImages(listdates.map(lambda summarize_day:
                pr.filterDate(ee.Date(summarize_day), ee.Date(summarize_day).advance(1, 'day'))
                  .sum().select(['total_precipitation'])
                  .copyProperties(pr.first()).setMulti({
                        'Date': ee.Date(summarize_day),
                        'system:time_start': ee.Date(summarize_day).millis()
                    })
            ))

            Sub_FeatureCollection_pr = Sub_FeatureCollection.map(lambda feature: extract_feature(feature, dailyDataset)).flatten()
            Sub_FeatureCollection_pr = Sub_FeatureCollection_pr.select(['.*'], None, False)

            # Output configuration
            description = f'NLDAS_PRC_Batch_{i}_{filterdate}'
            output_folder = '001_Project_Exports'
            print(description)

            # Export as CSV to Google Drive
            task = ee.batch.Export.table.toDrive(
                collection=Sub_FeatureCollection_pr,
                description=description,
                folder=output_folder,
                fileNamePrefix=description,
                fileFormat='CSV'
            )
            task.start()

        break  # Exit the loop after processing the target batch
print("Done!")

29
NLDAS_PRC_Batch_9_2019
Done!
